In [23]:
#
#
#Author: WU Nan
#Date: 2016.12.5
#Description: This is the csv to csv programme file to 
#Further target: 
#1) should calulate "whether it is Normal Distribution or not "  
#2) Give data visualization
#Give the very normal risk ranking
#
#
#
#
from __future__ import print_function
from elasticsearch import Elasticsearch
import sys
from operator import add
from pyspark import SparkContext
from pyspark.mllib.regression import LinearRegressionWithSGD, LabeledPoint
from pyspark.mllib.classification import SVMWithSGD, SVMModel

import numpy as np
import csv
import math

#period: the days of period you choose
#✅WELL DONE
def get_csv_data(filename):
    filename = "file:/Users/nancywu/sparkhadoop/datatest/"+filename+".csv"
    File = sc.textFile(filename)
    File.map(lambda line: line.split(","))
    File.filter(lambda line: len(line) > 0)
    File.map(lambda line: (line[0], line[1]))
    data = File.collect()
    stock_text = [d.split(",") for d in data]

    #From backward of Period of present date (90 days before 2016.4.1) to present date 2016.4.1
    #historical period you want to choose
    period = 365*3

    open_price = [round(float(stock_text[i][1]),32) for i in range(period,0,-1)]
    close_price = [round(float(stock_text[i][4]),32) for i in range(period,0,-1)]
    volume=[long(stock_text[i][5]) for i in range (period, 0,-1)]
    date=["Date"]
    [date.append(stock_text[i][0]) for i in range(period,0,-1)]

    print ("get_csv_data: done")
    #print ( open_price,close_price,S0,True_price)
    return open_price,close_price,volume,date

#mu: the mean of sample training size
#v:
#NUMPY WRONG!!solved it already on Nov.28   ✅

def risk_assessment_price_change (filename):
    open_price,close_price, volume, date= get_csv_data(filename)
    
    risk_assessment_initial = list(map(lambda x: round((x[0]-x[1]),1), zip(open_price, close_price)))
    print("====risk assessment initial======")
    tmp=sc.parallelize(risk_assessment_initial)
    Change_initial = tmp.map(lambda distance: (distance, 1))\
            .reduceByKey(lambda a, b: a + b)
    Change_list=Change_initial.collect()
    
    print ("==========Change list=======================")
    print(Change_list)
    return Change_list

def generation_output(True_price,prediction, Date):
    #Date=
    output = []
    for i in range(1,len(Date)-2):
        #features=训练集,这里可以自己调整去做尝试； label=target目标值
        tmp = LabeledPoint(label=True_price[i],features=[prediction[i],prediction[i+1],prediction[i+2]])
        output.append(tmp)
    
    return output_model

#Do not  test it yet⚠️
def writeToElastic(fileindex,es,filename,stock_text):
    df=stock_text
    j = 1
    actions = []
    count = int(len(df))
    while (j < count):
        action = {
                   "_index": fileindex, # 这里不可以是大写，都是小写
                   "_type": filename,
                   "_id": j,
                   "_source": {
                               "date":df[j][0],
                               "open":float(df[j][1]),
                               "high":float(df[j][2]),
                               "low":float(df[j][3]),
                               "close":float(df[j][4]),
                               "volume":int(df[j][5]),
                               "adjClose":float(df[j][6]),
                               #"timestamp": datetime.now()
                                }
                   }
        print(action)
        actions.append(action)
        j += 1
        if (len(actions) == 180):
            helpers.bulk(es, actions)
            del actions[0:len(actions)]
            
    if (len(actions) >0 ):
            helpers.bulk(es, actions)
            del actions[0:len(actions)]
    

    
    
        
if __name__ =="__main__":
    #sc = SparkContext(appName="Monte Carlo")
    Ticker = sc.textFile("file:/Users/nancywu/sparkhadoop/datatest/Tickertest.csv")
    filelist = Ticker.map(lambda f: f.split(",")).collect()
    #l = Ticker.collect()
    #filelist = l[0].split(",")
    print(filelist)
    es = Elasticsearch()
    print("===========start============")
    for f in filelist:
        try:
            name = f[0]+".csv"
            print (name,"simulation",f[0])
            change_list = risk_assessment_price_change(f[0])
            print("=====risk assessment pass=====")
            print(len(change_list))
            #output = generation_output(True_price,prediction, Date)
            print("=======output pass=======")
            print (name)
            sc.parallelize(change_list).repartition(1).saveAsTextFile("file:/Users/nancywu/sparkhadoop/datatest_result/" + name)
            #writeToElastic("predictvalue",es,name,output)
        except:
            import traceback
            traceback.print_exc()
            print("No service for this stock")





[[u'AAPL']]
===========start============
AAPL.csv simulation AAPL
get_csv_data: done
====risk assessment initial======
==========Change list=======================
[(-5.5, 4), (7.5, 4), (6.8, 3), (0.5, 22), (-10.6, 1), (8.0, 2), (17.7, 1), (9.5, 1), (7.8, 3), (2.4, 4), (-3.4, 6), (4.5, 3), (-8.6, 1), (-3.9, 4), (-2.9, 7), (4.0, 1), (-9.6, 2), (0.0, 21), (-12.0, 1), (3.5, 2), (-0.5, 10), (11.5, 1), (2.9, 6), (-13.1, 1), (17.5, 1), (6.5, 4), (-9.1, 1), (8.5, 1), (5.5, 3), (-0.3, 19), (6.3, 2), (4.3, 9), (-4.8, 4), (7.3, 2), (-6.5, 2), (-8.1, 3), (-7.3, 3), (-11.5, 1), (-2.4, 9), (-9.5, 1), (-10.5, 2), (-1.5, 18), (1.5, 14), (-7.8, 1), (4.8, 5), (12.6, 2), (-5.8, 3), (-6.8, 1), (0.3, 18), (0.6, 8), (11.6, 4), (-0.6, 22), (8.6, 2), (-10.1, 2), (-8.5, 1), (-5.3, 5), (1.7, 10), (-15.1, 1), (-4.3, 4), (-2.5, 7), (2.5, 9), (10.5, 2), (-13.6, 1), (-4.0, 2), (-3.5, 3), (9.1, 1), (1.2, 9), (-1.7, 5), (21.2, 1), (10.6, 1), (11.1, 1), (-1.2, 14), (-8.0, 3), (5.3, 3), (3.9, 5), (-7.5, 2), (5.8, 6), 

Traceback (most recent call last):
  File "<ipython-input-23-6963714e37cf>", line 132, in <module>
    sc.parallelize(change_list).repartition(1).saveAsTextFile("file:/Users/nancywu/sparkhadoop/datatest_result/" + name)
  File "/Users/nancywu/sparkhadoop/python/pyspark/rdd.py", line 1506, in saveAsTextFile
    keyed._jrdd.map(self.ctx._jvm.BytesToString()).saveAsTextFile(path)
  File "/Users/nancywu/sparkhadoop/python/lib/py4j-0.9-src.zip/py4j/java_gateway.py", line 813, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/Users/nancywu/sparkhadoop/python/pyspark/sql/utils.py", line 45, in deco
    return f(*a, **kw)
  File "/Users/nancywu/sparkhadoop/python/lib/py4j-0.9-src.zip/py4j/protocol.py", line 308, in get_return_value
    format(target_id, ".", name), value)
Py4JJavaError: An error occurred while calling o1348.saveAsTextFile.
: org.apache.hadoop.mapred.FileAlreadyExistsException: Output directory file:/Users/nancywu/sparkhadoop/datatest_result/AAPL.